In [2]:
import numpy as np
import pickle
import random
import matplotlib.pyplot as plt
import tensorflow as tf

def unpickle(file):
    fo = open(file, 'rb')
    dict = pickle.load(fo, encoding='latin1')
    fo.close()
    return dict

def show_some_examples(names, data, labels):
    plt.figure()
    rows, cols = 4,4
    random_idxs = random.sample(range(len(data)), rows * cols)
    for i in range(rows * cols):
        plt.subplot(rows, cols, i + 1)
        j = random_idxs[i]
        plt.title(names[labels[j]])
        img = np.reshape(data[j, :], (24, 24))
        plt.imshow(img, cmap='Greys_r')
        plt.axis('off')
    plt.tight_layout()
    #plt.savefig('cifar_examples.png')

def clean(data):
    imgs = data.reshape(data.shape[0], 3, 32, 32)
    grayscale_imgs = imgs.mean(1)
    cropped_imgs = grayscale_imgs[:, 4:28, 4:28]
    img_data = cropped_imgs.reshape(data.shape[0], -1)
    img_size = np.shape(img_data)[1]
    means = np.mean(img_data, axis = 1)
    meansT = means.reshape(len(means), 1)
    stds = np.std(img_data, axis = 1)
    stdsT = stds.reshape(len(stds), 1)
    adj_stds = np.maximum(stdsT, 1.0 / np.sqrt(img_size))
    normalized = (img_data - meansT) / adj_stds
    return normalized

def read_data():
    names = unpickle('./data/cifar-10-batches-py/batches.meta')['label_names']
    print('names: ', names)
    
    data, labels = [], []
    for i in range(1, 6):
        filename = './data/cifar-10-batches-py/data_batch_' + str(i)
        batch_data = unpickle(filename)
        if len(data) > 0:
            data = np.vstack((data, batch_data['data']))
            labels = np.hstack((labels, batch_data['labels']))
        else:
            data = batch_data['data']
            labels = batch_data['labels']
   
    print('data shape: {},  labels shape {}'.format(np.shape(data), np.shape(labels)))
    
    data = clean(data)
    data = data.astype(np.float32)
    return names, data, labels

names, data, labels = read_data()
show_some_examples(names, data, labels)

W = tf.Variable(tf.random.normal([5, 5, 1, 32]))

def show_weights(W):
    plt.figure()
    rows, cols = 4, 8
    for i in range(np.shape(W)[3]):
        img = W[:, :, 0, i]
        plt.subplot(rows, cols, i + 1)
        plt.imshow(img, cmap = 'Greys_r', interpolation='none')
        plt.axis('off')
    plt.show()
    
def show_conv_results(data):
    plt.figure()
    rows, cols = 4, 8
    for i in range(np.shape(data)[3]):
        img = data[0, :, :, i]
        plt.subplot(rows, cols, i + 1)
        plt.imshow(img, cmap = 'Greys_r', interpolation='none')
        plt.axis('off')
    plt.show()
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    W_val = sess.run(W)
    show_weights(W_val)
        

names:  ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
data shape: (50000, 3072),  labels shape (50000,)
Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
raw_data = data[4, :]
raw_img = np.reshape(raw_data, (24, 24))
plt.figure()
plt.imshow(raw_img, cmap = 'Greys_r', interpolation='none')

x = tf.reshape(raw_data, shape=[-1, 24, 24, 1])
b = tf.Variable(tf.random.normal([32]))
conv = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
conv_with_b = tf.nn.bias_add(conv, b)
conv_out = tf.nn.relu(conv_with_b)

k = 2
maxpool = tf.nn.max_pool(conv_out, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    conv_val = sess.run(conv)
    show_conv_results(conv_val)
    
    conv_out_val = sess.run(conv_out)
    show_conv_results(conv_out_val)
    
    maxpool_val = sess.run(maxpool)
    show_conv_results(maxpool_val)
    print(np.shape(maxpool_val))

In [ ]:
def conv_layer(x, W, b):
    conv = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    conv_with_b = tf.nn.bias_add(conv, b)
    conv_out = tf.nn.relu(conv_with_b)
    return conv_out

def maxpool_layer(conv, k=2):
    return tf.nn.max_pool(conv, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')
    
names, data, labels = read_data()

x = tf.placeholder(tf.float32, [None, 24 * 24])
y = tf.placeholder(tf.float32, [None, len(names)])

W1 = tf.Variable(tf.random.normal([5, 5, 1, 64]))
b1 = tf.Variable(tf.random.normal([64]))

W2 = tf.Variable(tf.random.normal([5, 5, 64, 64]))
b2 = tf.Variable(tf.random.normal([64]))

W3 = tf.Variable(tf.random.normal([6 * 6 * 64, 1024]))
b3 = tf.Variable(tf.random.normal([1024]))

W_out = tf.Variable(tf.random.normal([1024, len(names)]))
b_out = tf.Variable(tf.random.normal([len(names)]))

x_reshape = tf.reshape(x, shape=[-1, 24, 24, 1])

conv_out1 = conv_layer(x_reshape, W1, b1)
maxpool_out1 = maxpool_layer(conv_out1)
norm1 = tf.nn.lrn(maxpool_out1, 4, bias=1.0, alpha=0.001/9.0, beta=0.75)

conv_out2 = conv_layer(norm1, W2, b2)
norm2 = tf.nn.lrn(maxpool_out1, 4, bias=1.0, alpha=0.001/9.0, beta=0.75)
maxpool_out2 = maxpool_layer(conv_out2)

maxpool_reshaped = tf.reshape(maxpool_out2, [-1, W3.get_shape().as_list()[0]])
local = tf.add(tf.matmul(maxpool_reshaped, W3), b3)
local_out = tf.nn.relu(local)

model_op = tf.add(tf.matmul(local_out, W_out), b_out)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model_op, labels=y))

train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

correct_pred = tf.equal(tf.argmax(model_op, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    onehot_labels = tf.one_hot(labels, len(names), on_value=1., off_value=0., axis=-1)
    onehot_vals = sess.run(onehot_labels)
    batch_size = len(data) // 200
    print('batch size:', batch_size)
    for j in range(0, 100):
        for i in range(0, len(data), batch_size):
            batch_data = data[i:i + batch_size, :]
            batch_onehot_vals = onehot_vals[i:i + batch_size, :]
            _, accuracy_val = sess.run([train_op, accuracy], feed_dict={x: batch_data, y:batch_onehot_vals})
        if j % 10 == 0:
            print(j, accuracy_val)

names:  ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
data shape: (50000, 3072),  labels shape (50000,)
batch size: 250
49750 0.264
49750 0.112
49750 0.132
49750 0.136
49750 0.052
